In [1]:
#%% Importiere alle wichtigen libraries
import numpy as np               # library für Rechenoperationen
import scipy.constants as sc     # library für Naturkonstanten
import matplotlib.pyplot as plt  # library für das Erstellen von Plots

#%% Definiere Funktionen

# Definiere Translationsmatrix in der Form Matrix = np.matrix([[D_11, D_12],[D_21, D_22]])
def D(d,k):         
    Matrix = np.matrix([[np.exp(-1j*k*d),0],[0,np.exp(1j*k*d)]]) 
    return Matrix

# Definiere Matrix für Übergang an Potentialstufe in der Form Matrix = np.matrix([[T_11, T_12],[T_21, T_22]])
def T(k1,k2):       
    Matrix = np.matrix([[(k1+k2)/2/k1, (k1-k2)/2/k1],[(k1-k2)/2/k1,(k1+k2)/2/k1]])
    return Matrix

# Definiere Potential in der Form np.array([[PotentialWert1 in eV, Länge1 in nm],[PotentialWert2 in eV, Länge2 in nm],...])
Potential = np.array([[-2,1],[3,0.3],[0,1]])

# GesamtEnergie in eV
GesEnergie = 2

# Setze den Anfangszustand des Teilchens in der Form np.array([[a_n],[b_n]])
AnfangsZustand = np.array([[1],[0]])

# Setze Masse des Teilchens in kg
Mass = sc.m_e


# Definiere Potentialgrenzen
PotAnfang = 0
PotGrenzen = np.cumsum(Potential[:,1])
PotGrenzen = np.insert(PotGrenzen,0,PotAnfang)

# x-Werte zum Plotten in nm
xVals = np.linspace(PotGrenzen[0],PotGrenzen[-1],1000)

# Berechne die Wellenzahlen
WellenZahl = np.sqrt((GesEnergie-Potential[:,0])*sc.e*2*Mass, dtype=complex)/sc.hbar/1e9

#%% Berechnung der Transmissions- und Reflexionswahrscheinlichkeit

TotTransferMatrix = np.eye(2)
for CPot in range(-1,-len(WellenZahl)-1,-1):
    if CPot == -len(WellenZahl):
        TotTransferMatrix = np.mat(D(Potential[CPot,1],WellenZahl[CPot]))*np.mat(TotTransferMatrix)
    else:
        TotTransferMatrix = np.mat(T(WellenZahl[CPot-1],WellenZahl[CPot]))*np.mat(D(Potential[CPot,1],WellenZahl[CPot]))*np.mat(TotTransferMatrix)
        
Endzustand = TotTransferMatrix*np.mat(AnfangsZustand)

PT = np.abs(AnfangsZustand[0]/Endzustand[0])**2*WellenZahl[2]/WellenZahl[0]
PR = np.abs(Endzustand[1]/Endzustand[0])**2

print('Reflexionswahrscheinlichkeit = ', PR)
print('Transmissionswahrscheinlichkeit = ', PT)
#%% Berechne Real- und Imaginärteil der Wellenfunktion und das Betragsquadrat




# Berechne Kumulierte Matrix D_0*T_01*D_1*...
CumMatrix = np.ones((len(WellenZahl)-1,2,2), dtype=complex)
for CWelle in range(-1,-len(WellenZahl),-1):
    if CWelle == -1:
        CumMatrix[CWelle,:,:] = np.mat(T(WellenZahl[CWelle-1],WellenZahl[CWelle]))*np.mat(D(Potential[CWelle,1],WellenZahl[CWelle]))
    else:
        CumMatrix[CWelle,:,:] = np.mat(T(WellenZahl[CWelle-1],WellenZahl[CWelle]))*np.mat(D(Potential[CWelle,1],WellenZahl[CWelle]))*np.mat(CumMatrix[CWelle+1,:,:])

# Definiere Funktionen für die verschiedenen Potentialbereiche
FuncList = []
CondList = []
for CWelle in range(len(WellenZahl)):
    
    if CWelle == len(WellenZahl)-1:
        FuncList.append(lambda x: np.sum(np.mat(D(PotGrenzen[CWelle+1]-x,WellenZahl[CWelle]))*np.mat(AnfangsZustand)))
        CondList.append(np.greater(xVals,PotGrenzen[CWelle]) & np.less_equal(xVals,PotGrenzen[CWelle+1]))
    elif CWelle == 0:
        FuncList.append(lambda x: np.sum(np.mat(D(PotGrenzen[CWelle+1]-x,WellenZahl[CWelle]))*np.mat(CumMatrix[CWelle,:,:])*np.mat(AnfangsZustand)))
        CondList.append(np.greater_equal(xVals,PotGrenzen[CWelle]) & np.less_equal(xVals,PotGrenzen[CWelle+1]))
    else:
        FuncList.append(lambda x: np.sum(np.mat(D(PotGrenzen[CWelle+1]-x,WellenZahl[CWelle]))*np.mat(CumMatrix[CWelle,:,:])*np.mat(AnfangsZustand)))
        CondList.append(np.greater(xVals,PotGrenzen[CWelle]) & np.less_equal(xVals,PotGrenzen[CWelle+1]))

# Berechne Wellenfunktion für die einzelnen Potentialabschnitte
PotKurve = np.zeros(len(xVals))
WaveFunc = np.zeros(len(xVals),dtype = complex)
for CWelle in range(len(FuncList)):
    WaveFunc[CondList[CWelle]] = np.squeeze(FuncList[CWelle](xVals[CondList[CWelle]]))
    PotKurve[CondList[CWelle]] = Potential[CWelle,0]

# Plot Ergebnis und speicher als pdf
f, (ax1, ax2) = plt.subplots(2,1,sharex='col')
ax1.plot(xVals,PotKurve, Color = 'k')
ax1.plot(xVals,np.real(WaveFunc),label = 'Realteil')
ax1.plot(xVals,np.imag(WaveFunc),label = 'Imaginärteil')
ax1.hlines(GesEnergie,np.min(xVals),np.max(xVals), Color = 'r',Linestyle = 'dashed',label = 'Gesamtenergie')
ax1.set_title('Real- und Imaginärteil')
ax1.set_ylim([-6, 4.5])
ax1.legend(loc = 'lower right')
ax2.plot(xVals,np.abs(WaveFunc)**2)
ax2.plot(xVals,PotKurve, Color = 'k')
ax2.set_title('Betragsquadrat')
ax2.set_xlabel('x [nm]')
f.savefig('PotentialStufe.pdf')

Reflexionswahrscheinlichkeit =  [[ 0.86756453]]
Transmissionswahrscheinlichkeit =  [[ 0.13243547+0.j]]
